In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/GRU #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f)
    return labels

In [5]:
training_data_y = labels("../Labels-Train.npy")
test_y = labels("../Labels-Test.npy")

training_data_x = pickle.load(open("TS-Train.pkl", "rb"))
test_x = pickle.load(open("TS-Test.pkl", "rb"))

print(training_data_x.columns[training_data_x.isna().any()].tolist())
print(test_x.columns[test_x.isna().any()].tolist())

training_data_x = training_data_x.dropna(axis=1)
test_x = test_x.dropna(axis=1)

print(training_data_x.shape, test_x.shape)

training_data_x = preprocessing.normalize(training_data_x,norm='max', axis=0)
test_x = preprocessing.normalize(test_x,norm='max', axis=0)

training_data_x = convert_3D(training_data_x)
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

[]
[]
(12589, 769) (1574, 769)


In [6]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save("keras-models/KerasGRU")
#model.save("kerasgru_h5.h5")

Epoch 1/10
276/276 [==============================] - 21s 61ms/step - loss: 0.6868 - accuracy: 0.7666 - auc: 0.4747 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_negatives: 2056.0000 - false_positives: 1.0000 - true_negatives: 6755.0000 - true_positives: 0.0000e+00 - val_loss: 0.6828 - val_accuracy: 0.7802 - val_auc: 0.4779 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 830.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 2947.0000 - val_true_positives: 0.0000e+00
Epoch 2/10
276/276 [==============================] - 15s 53ms/step - loss: 0.6800 - accuracy: 0.7667 - auc: 0.4575 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_negatives: 2056.0000 - false_positives: 0.0000e+00 - true_negatives: 6756.0000 - true_positives: 0.0000e+00 - val_loss: 0.6757 - val_accuracy: 0.7802 - val_auc: 0.4979 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 830.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 2947.

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [7]:
%tensorboard --logdir logs/GRU/fit